# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234349178175                   -0.50    8.0         
  2   -7.250191020042       -1.80       -1.40    1.0   8.66ms
  3   -7.250941857359       -3.12       -1.92    1.0   9.01ms
  4   -7.250915397455   +   -4.58       -1.85    2.0   11.1ms
  5   -7.251334408993       -3.38       -2.75    1.0   9.19ms
  6   -7.251337933762       -5.45       -3.17    1.0   8.87ms
  7   -7.251338665478       -6.14       -3.44    2.0   10.8ms
  8   -7.251338783390       -6.93       -3.94    1.0   9.11ms
  9   -7.251338795082       -7.93       -4.22    3.0   12.1ms
 10   -7.251338798205       -8.51       -4.90    1.0   9.48ms
 11   -7.251338798681       -9.32       -5.22    3.0   13.0ms
 12   -7.251338798699      -10.74       -5.55    1.0   9.55ms
 13   -7.251338798703      -11.42       -5.85    2.0   11.4ms
 14   -7.251338798704      -11.94       -6.16    1.0   9.62ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.0395008722016716
[ Info: Arnoldi iteration step 2: normres = 0.4310690884897886
[ Info: Arnoldi iteration step 3: normres = 0.650301280197362
[ Info: Arnoldi iteration step 4: normres = 0.6699777211481873
[ Info: Arnoldi iteration step 5: normres = 0.44523996063538746
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.43e-01, 7.39e-02, 3.39e-01, 2.38e-01, 2.60e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3194510949838267
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.15e-02, 1.81e-01, 2.30e-01, 9.42e-02, 4.21e-02)
[ Info: Arnoldi iteration step 7: normres = 0.06425540504906814
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.28e-03, 1.18e-02, 9.35e-03, 2.23e-02, 5.23e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11508493583551203
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (6.38e-05